In [1]:
import os
import shutil
import math
from glob import glob
from tqdm import tqdm
from PIL import Image

# ✅ Only these classes (remapped to 0–8)
CLASS_NAME_TO_ID = {
    "airplane": 0,
    "ship": 1,
    "storage-tank": 2,
    "ground-track-field": 3,
    "harbor": 4,
    "bridge": 5,
    "large-vehicle": 6,
    "small-vehicle": 7,
    "helicopter": 8,
}

def oriented_to_bbox(coords):
    xs = [float(coords[i]) for i in range(0, 8, 2)]
    ys = [float(coords[i]) for i in range(1, 8, 2)]
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    cx = (xmin + xmax) / 2
    cy = (ymin + ymax) / 2
    w = xmax - xmin
    h = ymax - ymin
    return cx, cy, w, h

def convert_label_file(dota_label_path, yolo_label_path, image_width, image_height):
    with open(dota_label_path, 'r') as f:
        lines = f.readlines()

    yolo_lines = []
    for line in lines:
        if line.strip() == "" or line.startswith("imagesource") or line.startswith("gsd"):
            continue
        parts = line.strip().split()
        if len(parts) < 9:
            continue

        coords = parts[:8]
        class_name = parts[8].lower()
        if class_name not in CLASS_NAME_TO_ID:
            continue
        class_id = CLASS_NAME_TO_ID[class_name]

        cx, cy, w, h = oriented_to_bbox(coords)

        # Normalize
        cx /= image_width
        cy /= image_height
        w /= image_width
        h /= image_height

        if w <= 0 or h <= 0:
            continue

        yolo_line = f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}"
        yolo_lines.append(yolo_line)

    if yolo_lines:
        with open(yolo_label_path, 'w') as f:
            f.write('\n'.join(yolo_lines))

def convert_split(split):
    dota_img_dir = f"DOTA/images/{split}"
    dota_label_dir = f"DOTA/labelTxt/{split}"
    yolo_img_dir = f"converted/images/{split}"
    yolo_label_dir = f"converted/labels/{split}"

    os.makedirs(yolo_img_dir, exist_ok=True)
    os.makedirs(yolo_label_dir, exist_ok=True)

    dota_images = glob(os.path.join(dota_img_dir, "*.png"))
    for img_path in tqdm(dota_images, desc=f"Converting {split}"):
        filename = os.path.basename(img_path).replace('.png', '')
        label_path = os.path.join(dota_label_dir, filename + ".txt")

        if not os.path.exists(label_path):
            continue

        # Copy image to new location
        shutil.copy(img_path, os.path.join(yolo_img_dir, filename + ".png"))

        # Get image size
        with Image.open(img_path) as img:
            w, h = img.size

        # Convert annotation
        yolo_label_path = os.path.join(yolo_label_dir, filename + ".txt")
        convert_label_file(label_path, yolo_label_path, w, h)

if __name__ == "__main__":
    convert_split("train")
    convert_split("val")


Converting val: 100%|████████████████████████████████████████████████████████████████| 458/458 [00:10<00:00, 42.21it/s]


In [2]:
import os
import shutil
import math
from glob import glob
from tqdm import tqdm

# Mapping of DOTA class names to class IDs
CLASS_NAME_TO_ID = {
    "airplane": 0,
    "ship": 1,
    "storage-tank": 2,
    "baseball-diamond": 3,
    "tennis-court": 4,
    "basketball-court": 5,
    "ground-track-field": 6,
    "harbor": 7,
    "bridge": 8,
    "large-vehicle": 9,
    "small-vehicle": 10,
    "helicopter": 11,
    "roundabout": 12,
    "soccer-ball-field": 13,
    "swimming-pool": 14,
}

# Convert oriented bounding box to axis-aligned bbox
def oriented_to_bbox(coords):
    xs = [float(coords[i]) for i in range(0, 8, 2)]
    ys = [float(coords[i]) for i in range(1, 8, 2)]
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    cx = (xmin + xmax) / 2
    cy = (ymin + ymax) / 2
    w = xmax - xmin
    h = ymax - ymin
    return cx, cy, w, h

def convert_label_file(dota_label_path, yolo_label_path, image_width, image_height):
    with open(dota_label_path, 'r') as f:
        lines = f.readlines()

    yolo_lines = []
    for line in lines:
        if line.strip() == "" or line.startswith("imagesource") or line.startswith("gsd"):
            continue
        parts = line.strip().split()
        if len(parts) < 9:
            continue

        coords = parts[:8]
        class_name = parts[8].lower()
        if class_name not in CLASS_NAME_TO_ID:
            continue
        class_id = CLASS_NAME_TO_ID[class_name]

        cx, cy, w, h = oriented_to_bbox(coords)

        # Normalize
        cx /= image_width
        cy /= image_height
        w /= image_width
        h /= image_height

        yolo_line = f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}"
        yolo_lines.append(yolo_line)

    with open(yolo_label_path, 'w') as f:
        f.write('\n'.join(yolo_lines))

def convert_split(split):
    dota_img_dir = f"DOTA/images/{split}"
    dota_label_dir = f"DOTA/labelTxt/{split}"
    yolo_img_dir = f"converted_All/images/{split}"
    yolo_label_dir = f"converted_All/labels/{split}"

    os.makedirs(yolo_img_dir, exist_ok=True)
    os.makedirs(yolo_label_dir, exist_ok=True)

    dota_images = glob(os.path.join(dota_img_dir, "*.png"))
    for img_path in tqdm(dota_images, desc=f"Converting {split}"):
        filename = os.path.basename(img_path).replace('.png', '')
        label_path = os.path.join(dota_label_dir, filename + ".txt")

        if not os.path.exists(label_path):
            continue

        # Copy image to new location
        shutil.copy(img_path, os.path.join(yolo_img_dir, filename + ".png"))

        # Get image size
        from PIL import Image
        with Image.open(img_path) as img:
            w, h = img.size

        # Convert annotation
        yolo_label_path = os.path.join(yolo_label_dir, filename + ".txt")
        convert_label_file(label_path, yolo_label_path, w, h)

if __name__ == "__main__":
    convert_split("train")
    convert_split("val")


Converting val: 100%|████████████████████████████████████████████████████████████████| 458/458 [00:06<00:00, 70.08it/s]
